**각종 설정하기**

In [10]:
# 의존성 패키지 설치
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 34.6 MB/s 
     |████████████████████████████████| 120 kB 66.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 


In [11]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [12]:
# 모델 환경 설정
import os
import torch
from glob import glob
from dataclasses import dataclass, field

@dataclass
class TrainArguments :
    pretrained_model_name: str = field(default="skt/kogpt2-base-v2")
    downstream_task_name: str = field(default="sentence-generation")
    downstream_model_dir: str = field(default="/content/drive/MyDrive/Do-it-자연어처리/checkpoint-story")
    max_seq_length: int = field(default=32)
    save_top_k: int = field(default=1)
    monitor: str = field(default="min val_loss")
    epochs: int = field(default=3)
    batch_size: int = field(default=96)
    learning_rate: float = field(default=5e-5)
    seed: int = field(default=None, metadata={"help": "random seed."})
    overwrite_cache: bool = field(default=False, metadata={"help": "Overwrite the cached training and evaluation sets"})
    force_download: bool = field(default=False,metadata={"help": "force to download downstream data and pretrained models."})
    test_mode: bool = field(default=False,metadata={"help": "Test Mode enables `fast_dev_run`"})
    cpu_workers: int = field(default=os.cpu_count(),metadata={"help": "number of CPU workers"})
    fp16: bool = field(default=False,metadata={"help": "Enable train on FP16"})

In [14]:
args = TrainArguments(
    max_seq_length = 32,
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    epochs=3,
    seed=7,
)

**토크나이저 준비하기**

In [ ]:
# 토크나이저 선언
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>', 
                                                    pad_token='<pad>', mask_token='<mask>')

print(tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o"))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']


**데이터 불러오기 / 전처리**

In [ ]:
# 데이터 로드
f_path = "/content/drive/MyDrive/Do-it-자연어처리/data/grimm/fairy_tale_clear.txt"

with open(f_path, "r", encoding="utf-8") as f :
    train = f.readlines()

train[0:10]

['옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.\n',
 '특히나 막내공주님이 아주 예뻤는데요, 어찌나 아름다운지, 해도 그녀의 얼굴에 햇볕을 비출 때마다 감탄을 하곤 하였더랬죠.\n',
 '왕의 성은 울창한 숲에 둘러싸여 있었어요.\n',
 '숲에 있는 나이 많은 라임 나무 아래에 우물이 하나 있었어요.\n',
 '화창한 날에, 막내공주님께서 숲으로 놀러 나와 시원한 분수 옆에 앉았어요.\n',
 '그녀는 따분할 땐 황금 공을 가져와 높이 던졌다가 다시 잡곤 했지요.\n',
 '황금 공놀이는 그녀가 가장 좋아하는 놀이었어요.\xa0\n',
 '그러다 우연히 공주의 작은 손으로 떠받치고 있던 황금 공이 바닥으로 떨어져 곧장 우물 속으로 굴러 들어갔지 뭐예요.\n',
 '공주님이 다급히 눈으론 공을 쫓았지만 허사였어요.\n',
 '우물은 꽤 깊었어요. 바닥이 보이지 않을 정도로요.\n']

In [ ]:
# 데이터셋 함수
class StoryDataset(Dataset) : 
    def __init__(self) :
        
    def __len__(self) :
        return 
    def __getitem__(self, index) :
        return self.lines[index]

# Data split

In [ ]:
# DataLoader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_dataloader = DataLoader(train_dataset,
                              batch_size = args.batch_size,
                              sampler = RandomSampler(train_dataset, replacement=False),
                              drop_last = True,
                              num_workers = args.cpu_workers,)
val_dataloader = DataLoader(val_dataset,
                            batch_size = args.batch_size,
                            sampler = SequentialSampler(val_dataset, replacement=False),
                            drop_last = True,
                            num_workers = args.cpu_workers,)

**모델 초기화**

In [ ]:
# 모델 준비
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

**파인 튜닝**

In [ ]:
!pip install pytorch-lightning

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ExponentialLR

# Define Task Module
class StoryGenerator(pl.LightningModule) :
  def __init__(self,model, args) :
    super().__init__()
    self.model = model
    self.args = args

  def configure_optimizers(self) :
    optimizer = optim.Adam(self.parameters(), lr=1e-3)
    scheduler = ExponentialLR(optimizer, gamma=0.9)
    return {'optimizer' : optimizer,
            'scheduler' : scheduler,
    }

  def training_step(self, inputs, batch_idx) :
    outputs = self.model(**inputs)
    self.log("loss", outputs.loss, prog_bar=False, logger=True, on_step=True, on_epoch=False)
    return outputs.loss

  def validation_step(self, inputs, batch_idx) :
    outputs = self.model(**inputs)
    self.log("val_loss", outputs.loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
    return outputs.loss

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning import ModelCheckpoint

# Define Trainer function
def get_trainer(args,return_trainer_only=True) :
  ckpt_path = os.path.abspath(args.downstream_model_dir)
  os.makedirs(ckpt_path, exist_ok=True)

  # Checkpoint Callback
  checkpoint_callback = ModelCheckpoint(
      dirpath = ckpt_path, 
      filename = '{epoch}-{val_loss:.2f}',
      save_top_k = args.save_top_k,
      mode = args.monitor.split()[0] # "min"
      monitor = args.monitor.split()[1]  # "val_loss"
  )

  # Trainer
  trainer = Trainer(
      max_pochs = args.epochs,
      fast_dev_run = args.test_mode,
      num_sanity_val_steps = None if args.test_mode else 0,
      callbacks = [checkpoint_callback],
      default_root_dir = ckpt_path,

      # For GPU Setup
      deterministic=torch.cuda.is_available() and args.seed is not None,
      gpus = torch.cuda.device_count() if torch.cuda.is_available() else None,
      precision = 16 if args.fp16 else 32,
  )

  if return_trainer_only :
    return trainer
  else : 
    return checkpoint_callback, trainer

In [ ]:
# Model
task = StoryGenerator(model, args)

# Fine-tuning
trainer = get_trainer(args)
trainer.fit(
    task,
    train_dataloaders = train_dataloader,
    val_dataloaders = val_dataloader,
)